# 1. Introduction

## What is GraphQL?

GraphQL is a query language for APIs and a runtime for executing those queries with your existing data. Unlike REST, which relies on multiple endpoints for different data, GraphQL allows you to request exactly what you need in a single query.<br>

The main components to understand are **queries**, **mutations**, and **resolvers**. These form the **core of any GraphQL API**.

### 1. Queries
A query is used to **fetch data** from a GraphQL server. Unlike REST APIs, which often require multiple endpoints, GraphQL queries allow clients to request exactly the data they need in a single call.

- **Syntax**
GraphQL queries use a declarative syntax where you specify:

    - The type of data you want.
    - Any nested fields you require.

#### Example
Here’s an example of a query to fetch a book's title and author:

In [ ]:
query {
  books {
    title
    author
  }
}

#### Key Features

- **Nested Fields**: Fetch related data in a single query.

In [ ]:
query {
  books {
    title
    author
    publisher {
      name
      location
    }
  }
}

- **Arguments**: Specify filters to refine your results.

In [ ]:
query {
  bookByTitle(title: "1984") {
    title
    author
  }
}

- **Aliases**: Rename fields for client-side convenience.

In [ ]:
query {
  gatsby: bookByTitle(title: "The Great Gatsby") {
    title
    author
  }
}

- **Fragments**: Reuse common field selections.

In [ ]:
query {
  books {
    ...BookFields
  }
}

fragment BookFields on Book {
  title
  author
  publishedYear
}

### 2. Mutations
A mutation is used to **modify data** on the server. Mutations can perform actions like creating, updating, or deleting records.

- **Syntax**
Mutations are similar to queries but include input arguments to specify the data being modified.

#### Example
Adding a new book:

In [ ]:
mutation {
  addBook(title: "New Book", author: "John Doe", publishedYear: 2023) {
    book {
      title
      author
    }
    ok
  }
}

#### Key Features
- **Return Values**: Mutations can return fields to confirm success or fetch related data after the operation.
- **Arguments**: Required to define what is being changed.
- **Chaining**: Multiple mutations can be executed in a single request.

### 3. Resolvers
Resolvers are the functions that **process queries or mutations** and fetch the data or perform the requested operations. They act as a bridge between the schema and the actual data.

- **Resolver Types**
    - **Query Resolvers**: Handle data retrieval.
    - **Mutation Resolvers**: Handle data modifications.
    - **Field Resolvers**: Handle specific fields, often for derived or nested data.

- **Resolver Anatomy**<br>
Resolvers accept three main arguments:
    - **parent**: The result of the parent resolver. Useful for nested fields.
    - **info**: Metadata about the execution, including the schema and context.
    - **args**: Arguments provided by the query or mutation.

#### Example

- **Query Resolver**:

In [ ]:
class Query(graphene.ObjectType):
    books = graphene.List(Book)

    def resolve_books(self, info):
        # Fetch data from the database or other sources
        return [
            Book(title="The Great Gatsby", author="F. Scott Fitzgerald"),
            Book(title="1984", author="George Orwell"),
        ]

- **Mutation Resolver**:

In [ ]:
class AddBook(graphene.Mutation):
    class Arguments:
        title = graphene.String(required=True)
        author = graphene.String(required=True)

    ok = graphene.Boolean()
    book = graphene.Field(Book)

    def mutate(self, info, title, author):
        new_book = Book(title=title, author=author)
        # Save to database here
        ok = True
        return AddBook(book=new_book, ok=ok)

- **Field Resolver**:

In [ ]:
class Book(graphene.ObjectType):
    title = graphene.String()
    author = graphene.String()
    summary = graphene.String()

    def resolve_summary(parent, info):
        return f"{parent.title} by {parent.author}"

### 4. Subscriptions
A subscription in GraphQL is used to listen to real-time events or changes on the server. It allows clients to receive updates when data changes, without having to repeatedly query the server.

- **Syntax** <br>
Subscriptions are defined similarly to queries but are used to listen to specific events. They maintain an open connection to the server, receiving updates when the event occurs.

#### Example
A subscription to listen for real-time updates to an order status:
<br>
- **Example Use Case** <br>
In an e-commerce system, you could have a subscription to notify customers about price changes or order updates.

In [ ]:
subscription {
  orderStatusUpdated(orderId: "123") {
    status
    trackingNumber
  }
}

#### Key Features
- **Real-Time Dat**: Subscriptions push updates to clients as soon as they happen, making them ideal for live notifications and updates.
- **Maintains Persistent Connectio**: Unlike queries and mutations, which are one-time requests, subscriptions keep an open connection to the server for ongoing communication.
- **Event-Driven**: Subscriptions are typically tied to specific events or data changes (e.g., order status updates or stock changes).

#### Key Concepts
- **Events**: The events being listened to are defined on the server. When a relevant change happens (e.g., an order status update), the server pushes the update to all clients subscribed to that event.
- **Subscription Server**: Typically uses WebSockets or similar protocols (like Server-Sent Events) to handle the persistent connection and send real-time data to the client.

### 5. Differences Between Queries and Mutations

<table border="1" cellpadding="10" cellspacing="0">
  <thead>
    <tr>
      <th>Aspect</th>
      <th>Queries</th>
      <th>Mutations</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Purpose</td>
      <td>Retrieve data.</td>
      <td>Modify data (create, update, delete).</td>
    </tr>
    <tr>
      <td>Structure</td>
      <td>Declarative, focuses on fields.</td>
      <td>Requires input arguments.</td>
    </tr>
    <tr>
      <td>HTTP Method</td>
      <td>Typically GET (though POST is allowed).</td>
      <td>Typically POST.</td>
    </tr>
    <tr>
      <td>Idempotence</td>
      <td>Always idempotent (same result on repeat calls).</td>
      <td>Not idempotent (changes data).</td>
    </tr>
  </tbody>
</table>


### 6. Context in Resolvers
Resolvers can use a context object to share global data like:
- User authentication/authorization details.
- Database connections.
- Configurations.

#### Example of Context Usage:

In [ ]:
class Query(graphene.ObjectType):
    books = graphene.List(Book)

    def resolve_books(self, info):
        user = info.context.get('user')
        if not user.is_authenticated:
            raise Exception("Authentication required!")
        return BookModel.query.all()

### 7. Schema Integration
In GraphQL, schemas define the structure of your API, while resolvers fetch or manipulate the data.

In [ ]:
class Query(graphene.ObjectType):
    book_by_title = graphene.Field(Book, title=graphene.String(required=True))

    def resolve_book_by_title(self, info, title):
        return BookModel.query.filter_by(title=title).first()

class Mutation(graphene.ObjectType):
    add_book = AddBook.Field()

schema = graphene.Schema(query=Query, mutation=Mutation)

### Key Benefits of GraphQL Concepts

- **Queries**: Allow fine-grained control over requested data.
- **Mutations**: Enable flexible and efficient data manipulation.
- **Resolvers**: Separate data logic from schema, improving maintainability.
- **Single Endpoint**: Unlike REST, GraphQL consolidates operations into one endpoint, improving simplicity.

# Installation

In [ ]:
pip install flask flask-graphql graphene sqlalchemy

# 2. Setting Up Flask with GraphQL

## 2.1 Create a Flask App

In [ ]:
from flask import Flask
from flask_graphql import GraphQLView

app = Flask(__name__)

@app.route('/')
def index():
    return "Welcome to the GraphQL API!"

if __name__ == "__main__":
    app.run(debug=True)

## 2.2 Add GraphQL Support

GraphQL in Flask is enabled through the flask-graphql library and graphene.

# 3. Defining a GraphQL Schema with Graphene

GraphQL schemas define the structure of your API. Use the graphene library to create types, queries, and mutations.

## 3.1 Define a Data Model
For this example, let’s manage a list of books with title, author, and published_year.

In [ ]:
import graphene

class Book(graphene.ObjectType):
    title = graphene.String()
    author = graphene.String()
    published_year = graphene.Int()

## 3.2 Create Queries
Define a query to fetch books.

In [ ]:
class Query(graphene.ObjectType):
    books = graphene.List(Book)

    def resolve_books(self, info):
        # Sample data
        return [
            Book(title="The Great Gatsby", author="F. Scott Fitzgerald", published_year=1925),
            Book(title="1984", author="George Orwell", published_year=1949),
        ]

## 3.3 Define the Schema
Combine your query into a schema.

In [ ]:
schema = graphene.Schema(query=Query)

## 3.4 Add GraphQLView
Integrate the GraphQL schema into Flask using GraphQLView.

In [ ]:
app.add_url_rule(
    '/graphql',
    view_func=GraphQLView.as_view(
        'graphql',
        schema=schema,
        graphiql=True  # Enables the GraphiQL interface
    )
)

Run the app and visit **/graphql** to use the GraphiQL interface.

# 4. Adding Mutations
Mutations allow clients to modify data.

## 4.1 Define a Mutation
Add a mutation to add new books.

In [ ]:
class AddBook(graphene.Mutation):
    class Arguments:
        title = graphene.String(required=True)
        author = graphene.String(required=True)
        published_year = graphene.Int(required=True)

    ok = graphene.Boolean()
    book = graphene.Field(lambda: Book)

    def mutate(self, info, title, author, published_year):
        new_book = Book(title=title, author=author, published_year=published_year)
        ok = True
        return AddBook(book=new_book, ok=ok)

## 4.2 Add Mutation to the Schema
Update the schema with the mutation.

In [ ]:
class Mutation(graphene.ObjectType):
    add_book = AddBook.Field()

schema = graphene.Schema(query=Query, mutation=Mutation)

Now you can add books using the add_book mutation.

# 5. Integrating with a Database (SQLAlchemy)

## 5.1 Set Up SQLAlchemy
Configure the Flask app:

In [ ]:
from flask_sqlalchemy import SQLAlchemy

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'
db = SQLAlchemy(app)

## 5.2 Create a Database Model
Define the database structure.

In [ ]:
class BookModel(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(80))
    author = db.Column(db.String(120))
    published_year = db.Column(db.Integer)

    def __repr__(self):
        return f'<Book {self.title}>'

## 5.3 Update Resolvers for the Database
Modify the query and mutation resolvers to interact with the database.

### Query Resolver:

In [ ]:
class Query(graphene.ObjectType):
    books = graphene.List(Book)

    def resolve_books(self, info):
        books = BookModel.query.all()
        return [
            Book(title=book.title, author=book.author, published_year=book.published_year)
            for book in books
        ]

### Mutation Resolver:

In [ ]:
class AddBook(graphene.Mutation):
    class Arguments:
        title = graphene.String(required=True)
        author = graphene.String(required=True)
        published_year = graphene.Int(required=True)

    ok = graphene.Boolean()
    book = graphene.Field(lambda: Book)

    def mutate(self, info, title, author, published_year):
        new_book = BookModel(title=title, author=author, published_year=published_year)
        db.session.add(new_book)
        db.session.commit()
        ok = True
        return AddBook(book=new_book, ok=ok)

# 6. Advanced Features

## 6.1 Filtering
You can add filtering to queries using arguments.

In [ ]:
class Query(graphene.ObjectType):
    book_by_title = graphene.Field(Book, title=graphene.String(required=True))

    def resolve_book_by_title(self, info, title):
        book = BookModel.query.filter_by(title=title).first()
        if book:
            return Book(title=book.title, author=book.author, published_year=book.published_year)
        return None

## 6.2 Pagination
Use libraries like graphene-sqlalchemy for easier pagination with SQLAlchemy.

## 6.3 Subscriptions
Subscriptions enable real-time updates but require additional libraries and WebSocket support (e.g., Flask-SocketIO).

# Full Example

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_graphql import GraphQLView
import graphene

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'
db = SQLAlchemy(app)

# Database Model
class BookModel(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(80))
    author = db.Column(db.String(120))
    published_year = db.Column(db.Integer)

    def __repr__(self):
        return f'<Book {self.title}>'

# GraphQL Types and Resolvers
class Book(graphene.ObjectType):
    title = graphene.String()
    author = graphene.String()
    published_year = graphene.Int()

class Query(graphene.ObjectType):
    books = graphene.List(Book)

    def resolve_books(self, info):
        books = BookModel.query.all()
        return [
            Book(title=book.title, author=book.author, published_year=book.published_year)
            for book in books
        ]

class AddBook(graphene.Mutation):
    class Arguments:
        title = graphene.String(required=True)
        author = graphene.String(required=True)
        published_year = graphene.Int(required=True)

    ok = graphene.Boolean()
    book = graphene.Field(lambda: Book)

    def mutate(self, info, title, author, published_year):
        new_book = BookModel(title=title, author=author, published_year=published_year)
        db.session.add(new_book)
        db.session.commit()
        ok = True
        return AddBook(book=new_book, ok=ok)

class Mutation(graphene.ObjectType):
    add_book = AddBook.Field()

schema = graphene.Schema(query=Query, mutation=Mutation)

# GraphQL Endpoint
app.add_url_rule(
    '/graphql',
    view_func=GraphQLView.as_view(
        'graphql',
        schema=schema,
        graphiql=True  # Enables the GraphiQL interface
    )
)

if __name__ == "__main__":
    db.create_all()
    app.run(debug=True)